Importing modules

In [15]:
import numpy as np
import pandas as pd

Defining custom functions

In [16]:
def display_columns():
  print("Number of columns: " + str(len(df.columns)))
  print(df.columns)

In [17]:
def display_summary():
  columns=list(df.columns)
  columns_cnt=len(columns)
  rows_cnt=len(df.index)

  print("Number of rows: " + str(rows_cnt))
  print("Number of columns: " + str(columns_cnt))
  print("Used columns: ")
  print(columns)

In [18]:
def rename_columns():
  new_headers = {
      # Closures & containment
      "C1M_School closing" : "C1M",
      "C2M_Workplace closing" : "C2M",
      "C3M_Cancel public events" : "C3M",
      "C4M_Restrictions on gatherings" : "C4M",
      "C5M_Close public transport" : "C5M",
      "C6M_Stay at home requirements" : "C6M",
      "C7M_Restrictions on internal movement" : "C7M",
      "C8EV_International travel controls" : "C8M",

      # Health measures
      "H1_Public information campaigns" : "H1"
  }

  df.rename(columns=new_headers, inplace=True)

In [19]:
def filter_columns():
  # Dropping not used columns with region information & other
  region_related_cls=["CountryCode", "RegionName", "RegionCode", "Jurisdiction", "M1_Wildcard", "StringencyIndex_Average_ForDisplay"]
  df.drop(columns=region_related_cls, inplace=True)

  # Defining regexes to filter not used columns indices
  economic_filter='^E[1-4]_*'
  health_system_filter='^H[2-8][M]?_*'
  vaccination_filter='^V[1-4][A-Z]?_*'
  GRI_filter='GovernmentResponseIndex*'
  CHI_filter='ContainmentHealthIndex*'
  ESI_filter='EconomicSupportIndex*'
  filters=[economic_filter, health_system_filter, vaccination_filter, GRI_filter, CHI_filter, ESI_filter]

  for filter in filters:
    df.drop(columns=df.filter(regex=filter), inplace=True)

In [20]:
def add_previous_stringency_index():
    df['Prev_StringencyIndex_Average']=0

    for i in range(1, len(df)):
      # First row from certain country
      if df.at[i, 'CountryName'] != df.at[i-1, 'CountryName']:
        df.loc[i, 'Prev_StringencyIndex_Average'] = 0.0
      else:
        df.loc[i, 'Prev_StringencyIndex_Average'] = df.loc[i-1, 'StringencyIndex_Average']

In [21]:
def get_npi_difference():
  # Add column for Stringency Index diff value
  df['Daily_StringencyIndex_Change']=0

  for i in range(1, len(df)):
    # Create np arrays for calculating Stringency Index diff
    npis=np.zeros((1,0))
    npis_flag=np.zeros((1,0))

    # First row from certain country
    if df.at[i, 'CountryName'] != df.at[i-1, 'CountryName']:

      for npi in subset:
        npis = np.append(npis, [df.at[i, npi]])
      for flag in flag_subset:
        npis_flag = np.append(npis_flag, [df.at[i, flag]])
    
    else:
      # Check whether any NPI value has changed
      for npi in subset:
        if df.at[i-1, npi] != df.at[i, npi]:
          npis = np.append(npis, [df.at[i, npi]])
        else:
          npis = np.append(npis, [0])

      # Check whether any NPI flag has changed
      for flag in flag_subset:
        if df.at[i-1, flag] != df.at[i, flag]:
          npis_flag = np.append(npis_flag, [df.at[i, flag]])
        else: 
          npis_flag = np.append(npis, [0])

      daily_si=calculate_simplified_si(npis)
      df.at[i, 'Daily_StringencyIndex_Change'] = daily_si


In [22]:
# Do not consider flags while calculating Stringency Index
def calculate_simplified_si(npis):
  # Max values of npis
  max_npi_values=[3,3,2,4,2,3,2,4,2]                                    
  si=0

  for i in range(0, len(npis)):
    si = si + (npis[i] / max_npi_values[i])
    
  si=si/len(npis) 
  return si

In [23]:
def calculate_si():
  pass

**[STEP 1]** Read data from CSV

In [24]:
path="./OxCGRT_data/OxCGRT_nat_latest.csv"

df=pd.read_csv(path)

display_summary()

Number of rows: 202819
Number of columns: 61
Used columns: 
['CountryName', 'CountryCode', 'RegionName', 'RegionCode', 'Jurisdiction', 'Date', 'C1M_School closing', 'C1M_Flag', 'C2M_Workplace closing', 'C2M_Flag', 'C3M_Cancel public events', 'C3M_Flag', 'C4M_Restrictions on gatherings', 'C4M_Flag', 'C5M_Close public transport', 'C5M_Flag', 'C6M_Stay at home requirements', 'C6M_Flag', 'C7M_Restrictions on internal movement', 'C7M_Flag', 'C8EV_International travel controls', 'E1_Income support', 'E1_Flag', 'E2_Debt/contract relief', 'E3_Fiscal measures', 'E4_International support', 'H1_Public information campaigns', 'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing', 'H4_Emergency investment in healthcare', 'H5_Investment in vaccines', 'H6M_Facial Coverings', 'H6M_Flag', 'H7_Vaccination policy', 'H7_Flag', 'H8M_Protection of elderly people', 'H8M_Flag', 'M1_Wildcard', 'V1_Vaccine Prioritisation (summary)', 'V2A_Vaccine Availability (summary)', 'V2B_Vaccine age eligibility/availability 

**[STEP 2]** Deleting not used and irrelevant columns

In [25]:
filter_columns()

**[STEP 3]** Add Continent column

In [26]:
continents_df = pd.read_csv("./OxCGRT_data/Countries-Continents.csv")
continents_dict = dict(zip(continents_df.Country, continents_df.Continent))
df['Continent'] = df['CountryName'].map(continents_dict)
continent_col = df.pop('Continent')
df.insert(0,'Continent',continent_col)

**[STEP 4]** Drop rows with NPI=Nan

In [27]:
df = df[df['StringencyIndex_Average'].notna()]

**[STEP 5]** Rename columns to improve readibility

In [28]:
rename_columns()

**[STEP 6]** Deleting rows where none of NPIs are defined

In [29]:
subset=["C1M", "C2M", "C3M", "C4M", "C5M", "C6M", "C7M", "C8M", "H1"]
flag_subset=["C1M_Flag", "C2M_Flag", "C3M_Flag", "C4M_Flag", "C5M_Flag", "C6M_Flag", "C7M_Flag", "H1_Flag"]
df.dropna(subset=flag_subset, how='all', inplace=True)
df.fillna(0, inplace=True)
df.reset_index(drop=True, inplace=True)

**[STEP 7]** Copy Stringency Index from previous day

In [30]:
add_previous_stringency_index()

**[STEP 8]** Save data with duplicates

In [31]:
df.to_csv('./OxCGRT_data/OxCGRT_clean_alldates.csv', encoding = 'utf-8-sig', index = False)

**[STEP 9]** Delete redundant rows

In [32]:
compared_columns=df.columns.tolist()

compared_columns.remove('Date')
compared_columns.remove('ConfirmedCases')
compared_columns.remove('ConfirmedDeaths')
compared_columns.remove('MajorityVaccinated')
compared_columns.remove('PopulationVaccinated')
compared_columns.remove('Prev_StringencyIndex_Average')

df.drop_duplicates(subset=compared_columns, inplace=True)
df.reset_index(drop=True, inplace=True)

**[STEP 10]** Calculate Stringency Index of NPI established certain day

In [33]:
get_npi_difference()

**[STEP 11]** Show results

In [34]:
df.head()
print(df['Daily_StringencyIndex_Change'].max())

0.8518518518518519


**[STEP 12]** Save DataFrame with results to csv

In [35]:
df.to_csv('./OxCGRT_data/OxCGRT_clean.csv', encoding = 'utf-8-sig', index = False)